This notebook explores the use synthetic data and Random Forest algorithm for forest stand classification.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

In [3]:
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

# Datasets
This notebook has random sampling of reference points.
The lightgbm and rf model is trained on all synthetic data and tested on all raw cloud free dates.

In [4]:
name_map = {'Larix decidua': 0,
 'Pinus sylvestris': 1,
 'Broadleaved trees': 2,
 'Picea abies': 3,
 'Pinus mugo': 4,
 'Pinus cembra': 5,
 'Abies alba': 6}
 # reverse the dictionary
value_map = {v: k for k, v in name_map.items()}

In [5]:
# load all features
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09','B11', 'B12', 'ndvi']
years = ["2018", "2019", "2020", "2021", "2022", "2023", "2024"]
DATA_PATH = "data/"

# Each TS different model

In [ ]:
# Preload and preprocess all raw and syn data
raw_data = {}
syn_data = {}

for year in years:
    raw_df = pd.read_csv(DATA_PATH + year + "_raw.csv", index_col=0, low_memory=False)
    raw_df.columns = ["band", "date", *[int(x) for x in raw_df.columns[2:]]]
    raw_df = raw_df.drop("class")
    raw_df.date = pd.to_datetime(raw_df.date)
    raw_df = raw_df.groupby(["band", "date"]).mean()
    raw_data[year] = raw_df

    syn_df = pd.read_csv(DATA_PATH + year + "_syn_raw.csv", index_col=0, low_memory=False)
    syn_df.columns = ["band", "date", *[int(x) for x in syn_df.columns[2:]]]
    y = syn_df.T.pop("class")[2:].astype(int)
    syn_df = syn_df.drop("class")
    syn_df.date = pd.to_datetime(syn_df.date)
    syn_df = syn_df.groupby(["band", "date"]).mean()
    syn_data[year] = (syn_df, y)

results = {}
for i in tqdm(range(5)):
    yearly_results = {}
    for year in years:   
        kf = KFold(n_splits=5, shuffle=True)
        
        org_df = raw_data[years[0]]  # Use a single raw_df to get shape
        df, y = syn_data[year]
        test_df_base = raw_data[year]

        for j, (train_index, test_index) in enumerate(kf.split(org_df.T[2:])): 
            test_df = test_df_base.loc[:, test_index]
            year_nan_flag = [False]  # a mutable list with one boolean

            def train_and_predict(row):
                row = row.dropna()
                train_df = df.loc[row.index, train_index].fillna(0)
                y_train = y[train_index]

                if train_df.isna().any().any():
                    if not year_nan_flag[0]:
                        print(year, "has", train_df.isna().any().sum(), "nan values")
                        year_nan_flag[0] = True
                    train_df = train_df.dropna(axis=1)
                    y_train = y[train_df.columns]

                model_rf = RandomForestClassifier(n_estimators=1000)
                model_rf.fit(train_df.T.values, y_train.values)
                return model_rf.predict(row.T.values.reshape(1, -1))[0]


            y_test = y[test_index]
            y_pred_rf = Parallel(n_jobs=-1)(
                delayed(train_and_predict)(row[1]) for row in test_df.T.iterrows()
            )

            acc_rf = 100 * metrics.accuracy_score(y_test, y_pred_rf)
            f1_w_rf = 100 * metrics.f1_score(y_test, y_pred_rf, average='weighted')
            yearly_results[f"syn_raw_rf_{j}"] = [f1_w_rf, acc_rf]

        results[f"{year}_{i}"] = yearly_results


  0%|          | 0/5 [00:00<?, ?it/s]

In [15]:

fold_averages = {}
for year_iter, yearly_data in results.items():
    year = year_iter.split('_')[0]
    iteration = year_iter.split('_')[1]
    if year not in fold_averages:
        fold_averages[year] = {}
    for fold, metrics in yearly_data.items():
        if fold not in fold_averages[year]:
            fold_averages[year][fold] = []
        fold_averages[year][fold].append(metrics)

# Calculate the mean across the iterations for each fold
for year in fold_averages:
    for fold, metrics_list in fold_averages[year].items():
        oa_values = [metrics[1] for metrics in metrics_list]
        f1_values = [metrics[0] for metrics in metrics_list]
        avg_oa = sum(oa_values) / len(oa_values)
        avg_f1 = sum(f1_values) / len(f1_values)  

        fold_averages[year][fold] = [avg_f1, avg_oa]



final_results = {}
for year, fold_data in fold_averages.items():
    oa_sum = 0
    f1_sum = 0
    num_folds = len(fold_data)
    for fold, (f1, oa) in fold_data.items():
        oa_sum += oa
        f1_sum += f1
    final_results[year] = {
        "OA": oa_sum / num_folds,
        "F1": f1_sum / num_folds
    }

df = pd.DataFrame.from_dict(final_results, orient='index')
df.index.name = "Year"
df.to_csv("rf_syn_raw_results.csv")

# 4. Print Average F1 and OA
avg_f1 = df["F1"].mean()
avg_oa = df["OA"].mean()

print(f"Average F1: {avg_f1}")
print(f"Average OA: {avg_oa}")

In [16]:
df.mean()

{'2018_0': {'_syn_raw_rf_4': [91.11969111969111, 90.92935794616466]},
 '2019_0': {'_syn_raw_rf_4': [85.71428571428571, 84.98989128281855]},
 '2020_0': {'_syn_raw_rf_4': [87.64478764478764, 87.26101849103453]},
 '2021_0': {'_syn_raw_rf_4': [87.25868725868726, 87.06825341851444]},
 '2022_0': {'_syn_raw_rf_4': [84.55598455598455, 83.87772927182291]},
 '2023_0': {'_syn_raw_rf_4': [83.01158301158301, 82.18682138862258]},
 '2018_1': {'_syn_raw_rf_4': [89.57528957528957, 89.41947039493104]},
 '2019_1': {'_syn_raw_rf_4': [88.03088803088804, 87.38942504599912]},
 '2020_1': {'_syn_raw_rf_4': [83.3976833976834, 82.77873263069388]},
 '2021_1': {'_syn_raw_rf_4': [88.03088803088804, 87.48069686128078]},
 '2022_1': {'_syn_raw_rf_4': [87.64478764478764, 87.0402007581986]},
 '2023_1': {'_syn_raw_rf_4': [81.85328185328186, 81.1665588202287]},
 '2018_2': {'_syn_raw_rf_4': [90.34749034749035, 90.2292896055127]},
 '2019_2': {'_syn_raw_rf_4': [86.10038610038609, 85.41992831317741]},
 '2020_2': {'_syn_raw_rf